In [1]:
import os
import cv2
import numpy as np
from albumentations import (
    Compose, Rotate, HorizontalFlip, VerticalFlip, RandomBrightnessContrast,
    HueSaturationValue, GaussianBlur, GaussNoise, RandomGamma, 
    PadIfNeeded, RandomCrop, Perspective, ToFloat
)

# Set your paths
input_dir = r"C:\Users\LENOVO\Downloads\Programs\positive"
output_dir = input_dir  # same directory

# Create augmentation pipeline
aug = Compose([
    Rotate(limit=90, p=0.5),
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    GaussianBlur(blur_limit=(1, 3), p=0.3),
    GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    RandomGamma(gamma_limit=(80, 120), p=0.3),
    Perspective(scale=(0.05, 0.1), keep_size=True, p=0.3),
], bbox_params={'format': 'yolo', 'min_area': 0, 'min_visibility': 0.3})

# Function to read YOLO annotations
def read_yolo_annotations(txt_path, img_width, img_height):
    with open(txt_path, 'r') as f:
        lines = f.readlines()
    bboxes = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        bboxes.append([x_center, y_center, width, height, class_id])
    return bboxes

# Function to write YOLO annotations
def write_yolo_annotations(txt_path, bboxes):
    with open(txt_path, 'w') as f:
        for bbox in bboxes:
            line = f"{int(bbox[4])} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n"
            f.write(line)

# Process each image
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load image and corresponding annotation
        img_path = os.path.join(input_dir, filename)
        base_name = os.path.splitext(filename)[0]
        txt_path = os.path.join(input_dir, f"{base_name}.txt")
        
        image = cv2.imread(img_path)
        if image is None:
            continue
            
        height, width = image.shape[:2]
        bboxes = read_yolo_annotations(txt_path, width, height)
        
        # Generate 15 augmented versions
        for i in range(1, 16):
            # Apply augmentation
            augmented = aug(image=image, bboxes=bboxes)
            aug_image = augmented['image']
            aug_bboxes = augmented['bboxes']
            
            # Save augmented image
            aug_img_name = f"{base_name}_aug{i}.jpg"
            aug_img_path = os.path.join(output_dir, aug_img_name)
            cv2.imwrite(aug_img_path, aug_image)
            
            # Save augmented annotations
            aug_txt_name = f"{base_name}_aug{i}.txt"
            aug_txt_path = os.path.join(output_dir, aug_txt_name)
            write_yolo_annotations(aug_txt_path, aug_bboxes)

print("Data augmentation completed successfully!")

C:\Users\LENOVO\AppData\Roaming\Python\Python313\site-packages\albumentations\check_version.py:107: UserWarning: Error fetching version info <urlopen error [Errno 11001] getaddrinfo failed>
  data = fetch_version_info()
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13872\1460218367.py:22: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  GaussNoise(var_limit=(10.0, 50.0), p=0.3),


Data augmentation completed successfully!
